Import librairies and packages

In [1]:
from tdmclient import ClientAsync, aw

import os
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import Vision.CamCalib as calib
import Vision.EnvTracker as tracker
import GlobalNav.DStar as d_star
import time
import cv2
import threading
import importlib
import Kalman

importlib.reload(d_star)
importlib.reload(tracker)

sys.path.insert(0, os.path.join(os.getcwd(), 'LocalNav/sensors'))
from local_occupancy import thymio_coords

sys.path.insert(0, os.path.join(os.getcwd(), 'plot'))
from plot import plot

sys.path.insert(0, os.path.join(os.getcwd(), 'NMPC'))
from nmpc import nmpc

sys.path.insert(0, os.path.join(os.getcwd(), 'LocalNav'))
sys.path.insert(0, os.path.join(os.getcwd(), 'GlobalNav'))
from LocalNav import localNav
#from GlobalNav import GlobalNav
%matplotlib tk

c:\Users\alexa\anaconda3\envs\mobile_robotics\lib\site-packages\do_mpc\sysid\__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
c:\Users\alexa\anaconda3\envs\mobile_robotics\lib\site-packages\do_mpc\opcua\__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


Initialize components & objects

In [2]:
#Start Thymio
USE_THYMIO = False

if USE_THYMIO:
    #Create a Client Object:
    client = ClientAsync()
    node = await client.wait_for_node()
    await node.lock()

    client.process_waiting_messages()

In [3]:
# Parameters
MAP_W, MAP_H = (120, 120)  # In cm

# Position
abs_pos = [[20, 20, 0]] # absolute x,y,theta coordinates of the robot
# Goal Position
goal_position = [[80, 80]]

# Initialize camCalib object to retrieve camera parameters
cam_calib = calib.CamCalib(None, (16,9))
cam_mat, dist_coefs, rvecs, tvecs = cam_calib.load_camera_params()
quit = False
run = True
wait_for_map = True
map_created = False

# Initialize the environment tracker object to create global map and track thymio
env_tracker = tracker.EnvTracker((MAP_W, MAP_H), cam_mat, dist_coefs)

map_global = np.zeros((MAP_W, MAP_H))


Main loop (run thymio)

In [4]:
# Open camera feed & initialize map
cv2.destroyAllWindows()
cv2.namedWindow("Thymio Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Thymio Tracking", 1680, 1050)
cam_feed = cv2.VideoCapture(0)
cam_feed.set(cv2.CAP_PROP_FRAME_WIDTH, 1680)
cam_feed.set(cv2.CAP_PROP_FRAME_HEIGHT, 1050)
if not cam_feed.isOpened():
    print("ERROR :: Could not open camera video feed!")
map_created, map_global, goal_pose, thymio_init_pose = env_tracker.wait_for_map(cam_feed, cam_calib, window_name="Thymio Tracking")
goal_position = np.array([goal_pose[:]])

plt.show()
thymio_init_pose = env_tracker.wait_for_thymio(cam_feed, cam_calib, "Thymio Tracking")
print("Thymio init pose: ", thymio_init_pose)
time.sleep(1)

# Switch map X,Y
map_global = np.transpose(map_global)

# GLOBAL PLANNING 
if np.sum(goal_pose) >= 0 and np.sum(thymio_init_pose) > 0:
    # If valid pose for goal and thymio
    best_path = d_star.FindGlobalPath(tuple(thymio_init_pose[:2].astype(int)), tuple(goal_pose.astype(int)), map_global.T)

# Adjustments before tracking
track_thymio = True
show_markers = False
frame_counter = 0
text_pose = 'POS: ??'
text_angle = 'ANGLE: ??'
text_vel = 'VEL: ??'
text_ang_vel = 'ANG. VEL: ??'
text_goal = f'GOAL: {np.round(goal_pose, 1)}'
pose_hist = np.zeros((2,10))
angle_hist = np.zeros(10)
cv2.resizeWindow("Thymio Tracking", 1200, 1000)

# Map is ready, run the thymio
plt.show()
quit = False
frame_count = 0
fig_main = plt.figure(3)
ax_main = fig_main.add_subplot(111)
goal_reached = False
L = []
T = []

optimal_state_estimate_k = np.array([0.0,0.0,0.0])
covariance_estimate_k = np.array([[0.1,0,0],[0,0.1,0],[0,0,0.1]]) #à modifier

L.append(optimal_state_estimate_k)
T.append(covariance_estimate_k)
controller_r = 0
controller_l = 0
next_point_index = 10
dest_point = np.array([[best_path[0,next_point_index], best_path[1,next_point_index]]])
print("DEST: ", dest_point)
while run and not quit:
    # -------------
    # VISION
    # ------------
    # Acquire cam image and perform thymio tracking
    success, frame = cam_feed.read()
    if not success:
        run = False
    # Detect map & thymio
    frame_corrected, roi = cam_calib.undistord(frame, cam_mat, dist_coefs)
    x,y,w,h = roi
    frame_corrected = frame_corrected[y:y+h, x:x+w, :]
    if track_thymio:
        success, thymio_pose, thymio_vel, img_thymio = env_tracker.updateThymio(frame_corrected, show_markers)
        if frame_count % 100:
            env_tracker.updateMapROI(frame_corrected)
        img_sidebar = np.zeros((img_thymio.shape[0], 200, 3), dtype=np.uint8)
        if success:
            abs_pos = np.array([thymio_pose[:]])
            text_pose = f'POSE: {np.round(thymio_pose[:2], 1)}'
            text_angle = f'ANGLE: {round(thymio_pose[2]*180/math.pi, 0)}'
            text_vel = f'VEL: {np.round(thymio_vel[:2], 1)}'
            text_ang_vel = f'ANG. VEL: {round(thymio_vel[2]*180/math.pi, 1)}'
        cv2.putText(img_sidebar, text_pose, (10, 100), cv2.FONT_HERSHEY_TRIPLEX, 0.5, [0,255,0], 1)
        cv2.putText(img_sidebar, text_angle, (10, 175), cv2.FONT_HERSHEY_TRIPLEX, 0.5, [0,255,0], 1)
        cv2.putText(img_sidebar, text_vel, (10, 250), cv2.FONT_HERSHEY_TRIPLEX, 0.5, [0,255,0], 1)
        cv2.putText(img_sidebar, text_ang_vel, (10, 325), cv2.FONT_HERSHEY_TRIPLEX, 0.5, [0,255,0], 1)
        cv2.putText(img_sidebar, text_goal, (10, 400), cv2.FONT_HERSHEY_TRIPLEX, 0.5, [0,255,0], 1)
        # Concatenate images
        img_thymio = cv2.hconcat([img_thymio, img_sidebar])
    else:
        img_thymio = env_tracker.getProjectedMap(frame_corrected)
    cv2.imshow("Thymio Tracking",img_thymio)
    key = cv2.waitKey(1)
    if key == ord('q'):
        run = False
    elif key == ord(' '):
        # Track thymio or not
        track_thymio = not track_thymio
    elif key == ord('m'):
        show_markers = True
    
    if USE_THYMIO:
        await node.wait_for_variables({"prox.horizontal"})
        prox_horizontal = list(node.v.prox.horizontal)
        #print(f'prox_hor: {prox_horizontal}')
    else:
        prox_horizontal = np.zeros(7)

    map_global = localNav(abs_pos, goal_position, prox_horizontal, map_global)
    # -----------
    # FILTERING
    # ----------
    #optimal_state_estimate_k,covariance_estimate_k = Kalman.Kalman(abs_pos[0],[0,0],L[-1],T[-1])
    #L.append(optimal_state_estimate_k)
    #T.append(covariance_estimate_k)
    # --------
    # MOTION
    # -------
    if frame_count % 30:
        controller_r, controller_l, states, time_points, controller = nmpc(abs_pos, dest_point, 0, 0, 0, 0)
    if math.sqrt( (abs_pos[0][0]-dest_point[0][0])**2 + (abs_pos[0][1]-dest_point[0][1])**2) > 6:
        v = {
        "motor.left.target": [int(controller_r) * 10],
        "motor.right.target": [int(controller_l) * 10],
        }
    else:
        goal_reached = True
        # GO to next point
        next_point_index += 5
        dest_point = np.array([[best_path[0,next_point_index], best_path[1][next_point_index]]])
        v = {
        "motor.left.target": [0],
        "motor.right.target": [0],
        }
    if USE_THYMIO:
        aw(node.set_variables(v))
    # -------
    # PLOTTING
    # -------
    if frame_count % 30:
        #clear_output(wait=True)  # Clear the previous plot
        ax_main.clear()
        plot(abs_pos, thymio_coords, goal_position, map_global, abs_pos[0][2], ax_main, best_path=best_path)
        fig_main.canvas.draw()
        fig_main.canvas.flush_events()
        #time.sleep(1)
    frame_count += 1

if cam_feed:
    cam_feed.release()
cv2.destroyAllWindows()
plt.close('all')

v = {
"motor.left.target": [0],
"motor.right.target": [0],
}
if USE_THYMIO:
    aw(node.set_variables(v))

Pixels per grid cell: 8.333333333333334 x 8.333333333333334
Goal detected @:  [107.64  86.13]
Thymio init pose:  [21.135 22.605  1.577]
DEST:  [[21 32]]


In [5]:
cam_feed.release()